In [2]:
# Celda 2 (nueva) - Agregar ANTES de la celda que descarga el dataset
from huggingface_hub import hf_hub_download

# Instalar pytesseract

# Descargar Tesseract OCR para Windows:
# https://github.com/UB-Mannheim/tesseract/wiki
# Instalar y agregar a PATH o configurar ruta en el script

In [4]:
# notebooks/descargar_facturas_con_ocr.py

"""
Script para descargar facturas del dataset de Hugging Face
y generar PDFs con texto seleccionable usando OCR
"""

import pandas as pd
import json
import io
from pathlib import Path
from PIL import Image
from huggingface_hub import hf_hub_download  # ← IMPORT FALTANTE
import pytesseract
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Configurar pytesseract (ajusta la ruta según tu instalación)
# En Windows, descarga Tesseract desde: https://github.com/UB-Mannheim/tesseract/wiki
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def create_searchable_pdf(image_path, pdf_path, ocr_text=None):
    """
    Crear PDF con imagen y capa de texto invisible (searchable)
    """
    image = Image.open(image_path)
    img_width, img_height = image.size
    
    # Crear PDF
    c = canvas.Canvas(str(pdf_path), pagesize=letter)
    page_width, page_height = letter
    
    # Calcular dimensiones manteniendo aspecto
    aspect = img_height / float(img_width)
    display_width = page_width - 40
    display_height = display_width * aspect
    
    if display_height > page_height - 40:
        display_height = page_height - 40
        display_width = display_height / aspect
    
    # Dibujar imagen
    c.drawImage(str(image_path), 20, page_height - display_height - 20,
               width=display_width, height=display_height)
    
    # Agregar texto OCR como capa invisible (si está disponible)
    if ocr_text:
        c.setFillColorRGB(0, 0, 0, alpha=0)  # Texto invisible
        c.setFont("Helvetica", 8)
        
        # Dividir texto en líneas y colocarlo
        lines = ocr_text.split('\n')
        y_pos = page_height - 40
        for line in lines[:50]:  # Limitar a 50 líneas
            if line.strip():
                c.drawString(30, y_pos, line[:100])  # Limitar longitud
                y_pos -= 12
            if y_pos < 40:
                break
    
    c.save()
    print(f"✅ PDF con texto seleccionable: {pdf_path.name}")

def main():
    print("=" * 60)
    print("📄 DESCARGADOR DE FACTURAS CON OCR")
    print("=" * 60)
    
    # Configuración
    repo_id = "katanaml-org/invoices-donut-data-v1"
    filename = "data/train-00000-of-00001-a5c51039eab2980a.parquet"
    facturas_dir = Path('../facturas_dataset')
    facturas_dir.mkdir(parents=True, exist_ok=True)
    
    # Descargar dataset
    print(f"\n📥 Descargando dataset de Hugging Face...")
    print(f"   Repositorio: {repo_id}")
    
    try:
        parquet_file = hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            repo_type="dataset"
        )
        print(f"✅ Archivo descargado")
        
        # Leer datos
        df = pd.read_parquet(parquet_file, engine='fastparquet')
        print(f"✅ Datos cargados: {len(df)} facturas")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return
    
    # Procesar facturas
    num_samples = min(50, len(df))
    print(f"\n💾 Procesando {num_samples} facturas...\n")
    
    for i in range(num_samples):
        try:
            row = df.iloc[i]
            
            if 'image.bytes' not in row:
                continue
            
            image_bytes = row['image.bytes']
            image = Image.open(io.BytesIO(image_bytes))
            
            # Guardar PNG
            png_path = facturas_dir / f"factura_{i+1}.png"
            image.save(png_path, 'PNG')
            print(f"[{i+1}/{num_samples}] ✅ Imagen guardada: {png_path.name}")
            
            # Extraer texto con OCR
            try:
                ocr_text = pytesseract.image_to_string(image, lang='eng+spa')
                print(f"           🔍 OCR extraído: {len(ocr_text)} caracteres")
            except Exception as ocr_error:
                print(f"           ⚠️  OCR no disponible: {ocr_error}")
                ocr_text = None
            
            # Crear PDF con texto seleccionable
            pdf_path = facturas_dir / f"factura_{i+1}.pdf"
            create_searchable_pdf(png_path, pdf_path, ocr_text)
            
            # Guardar metadata
            metadata = {
                "numero": i + 1,
                "ground_truth": row.get('ground_truth', ''),
                "image_path": row.get('image.path', ''),
                "ocr_available": ocr_text is not None,
                "archivos": {
                    "imagen": f"factura_{i+1}.png",
                    "pdf": f"factura_{i+1}.pdf"
                }
            }
            
            metadata_path = facturas_dir / f"factura_{i+1}_metadata.json"
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            
            print(f"           ✅ Metadata guardada\n")
            
        except Exception as e:
            print(f"❌ Error procesando factura {i+1}: {e}\n")
            continue
    
    print("\n" + "=" * 60)
    print("🎉 PROCESO COMPLETADO")
    print("=" * 60)
    print(f"📁 Facturas guardadas en: {facturas_dir.absolute()}")
    print(f"\n💡 RECOMENDACIÓN:")
    print(f"   - Usa los archivos PNG para mejor extracción con OpenAI Vision")
    print(f"   - Los PDFs tienen texto OCR si Tesseract está instalado")

if __name__ == "__main__":
    main()

📄 DESCARGADOR DE FACTURAS CON OCR

📥 Descargando dataset de Hugging Face...
   Repositorio: katanaml-org/invoices-donut-data-v1
✅ Archivo descargado
✅ Datos cargados: 425 facturas

💾 Procesando 50 facturas...

[1/50] ✅ Imagen guardada: factura_1.png
           ⚠️  OCR no disponible: tesseract is not installed or it's not in your PATH. See README file for more information.
✅ PDF con texto seleccionable: factura_1.pdf
           ✅ Metadata guardada

[2/50] ✅ Imagen guardada: factura_2.png
           ⚠️  OCR no disponible: tesseract is not installed or it's not in your PATH. See README file for more information.
✅ PDF con texto seleccionable: factura_2.pdf
           ✅ Metadata guardada

[3/50] ✅ Imagen guardada: factura_3.png
           ⚠️  OCR no disponible: tesseract is not installed or it's not in your PATH. See README file for more information.
✅ PDF con texto seleccionable: factura_3.pdf
           ✅ Metadata guardada

[4/50] ✅ Imagen guardada: factura_4.png
           ⚠️  OCR no di